# Homework 5

### Cassandra Bodin


Worked with: Mackenzie, Ryan Webster, Madison, Sean, Jimmy

In [68]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl

#import the read file 
from ReadFile import Read
from CenterOfMass import CenterOfMass

## Section 1-

In [80]:
##Part1
# Class to define mass profile of a given galaxy and simulation snapshot
class MassProfile:  
    
    def __init__(self, snap, galaxy):
        
        #Inputs:
        #snap, Snapshot number
        #galaxy, a string with the Galaxy name (ie MW, M31, M33)

        #add a string of the filenumber to the value "000"
        ilbl= '000' +str(snap)
        #remove all but the last 3 digits
        ilbl=ilbl[-3:]
        self.filename="%s_"%(galaxy) + ilbl + '.txt'
    
        # read data in the given file using Read
        self.time, self.total, self.data = Read(self.filename)                                                                                             


        # store the mass and positions of only the particles
        self.m = self.data['m']
        self.x = self.data['x']*u.kpc
        self.y = self.data['y']*u.kpc
        self.z = self.data['z']*u.kpc
        
        # store the galaxy name
        self.gname="%s_"%(galaxy)
     
    #Part 2
    #Function to calculate the mass enclosed within a radius
    def MassEnclosed(self,ptype,r):
        #Inputs:
        #ptype, the particle type
        #     Halo:1
        #     Disk:2
        #     Bulge:3
        #r, an array of radii, magnitude not vector quantities (kpc)
        
        #Returns:
        #mencl, array of masses enclosed in radius (Msun)
        
        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        self.m = self.data['m'][self.index]*1e10 #data in 1e10Msun
        self.x = self.data['x'][self.index]*u.kpc
        self.y = self.data['y'][self.index]*u.kpc
        self.z = self.data['z'][self.index]*u.kpc
        
        # read COM Position data in the given file using COM_P
        COM = CenterOfMass(self.filename,2)
        COMP=COM.COM_P(0.1) #note we are defining alpha=0.1
        
        # Test, MW COM Position is [-2.07  2.95 -1.45] kpc
        #print("COMP is",COMP)

        
        #initialize an empty array
        mencl= np.zeros(len(r))
        
        
        #define a radius
        R=((self.x-COMP[0])**2+(self.y-COMP[1])**2+(self.z-COMP[2])**2)**(1/2)
            
        #for loop to cycle through each radius inside
        for i in range(len(r)):
                    
            #select all particles within the reduced radius 
            #    starting from original x,y,z, m         
            index = np.where(R < r[i])
            mnew = self.m[index]
            
            #sum all of the masses within that radius
            mencl[i]=np.sum(mnew)
            
        return mencl*u.Msun
    
    ##Part3
    #Function to calculate the mass enclosed within a radius
    def MassEnclosedTotal(self,r):
        #Inputs:
        #r, a 1D array of radii (kpc)
        
        #Returns:
        #mtot, array of masses (Msun)
        
        
        #Define mass enclosed in terms of each of the particle types
        #     Halo:1
        #     Disk:2
        #     Bulge:3
        mhalo= self.MassEnclosed(1,r)
        mdisk= self.MassEnclosed(2,r)
        mbulge= self.MassEnclosed(3,r)
        
        #M33 has no bulge so take this into account with if statement
        if self.gname == 'M33':
            Mtotencl = mhalo+mdisk
        else:
            Mtotencl = mhalo+mdisk+mbulge
            
        return Mtotencl
    
    ##Part4
    #Note:used code from lab3
    #create a function that returns the Hernquist 1990 Mass profile
    def HernquistMass(r,a,Mhalo):
        #Inputs:
        #r, the distance from the galactic center (kpc)
        #a, the scale radius (kpc)
        #Mhalo, total mass of the dark matter halo (1e12*Msun)

        #Returns:
        #total darkmatter mass enclosed within radius r (Msun)

        return np.round(Mhalo*r**2/(a+r)**2,2)*1e12*u.Msun
    
    ##Part5
    #create a function to create an array of circular velocities 
    #for each galaxy component at the radius of the input array
    def CircularVelocityTotal(r):
        #Inputs:
        #r, a 1D array of radii (kpc)
        
        #Returns:
        #Vcirc, an array of the total circulaar velocity at each r (km/s)
        
        

#### Test Part 2!!!

In [78]:
MW=MassProfile(0,"MW") #initializes mass profile class for MW

r=np.arange(0.25,30.5,1.5)*u.kpc #set radii range
print("rvalues",r) #print

MW.MassEnclosed(1,r) #get enclosed halo masses at each r element

rvalues [ 0.25  1.75  3.25  4.75  6.25  7.75  9.25 10.75 12.25 13.75 15.25 16.75
 18.25 19.75 21.25 22.75 24.25 25.75 27.25 28.75 30.25] kpc


<Quantity [0.00000000e+00, 1.26395200e+09, 4.54232750e+09, 9.20315050e+09,
           1.60363910e+10, 2.36991000e+10, 3.14803045e+10, 4.07624520e+10,
           5.26909990e+10, 6.34740895e+10, 7.57581230e+10, 8.82396490e+10,
           1.02380112e+11, 1.15651608e+11, 1.29871068e+11, 1.44169525e+11,
           1.58941964e+11, 1.73477412e+11, 1.85563953e+11, 1.98756452e+11,
           2.12659924e+11] solMass>

#### Test Part 3!!!

In [81]:
MW=MassProfile(0,"MW") #initializes mass profile class for MW

r=np.arange(0.25,30.5,1.5)*u.kpc #set radii range
print("rvalues",r) #print

MW.MassEnclosedTotal(r) #get enclosed halo masses at each r element

rvalues [ 0.25  1.75  3.25  4.75  6.25  7.75  9.25 10.75 12.25 13.75 15.25 16.75
 18.25 19.75 21.25 22.75 24.25 25.75 27.25 28.75 30.25] kpc


<Quantity [1.59640000e+09, 2.57516420e+10, 4.12176745e+10, 5.40637945e+10,
           6.72522775e+10, 7.98531960e+10, 9.18075315e+10, 1.05159795e+11,
           1.20759444e+11, 1.34975610e+11, 1.50047710e+11, 1.65038278e+11,
           1.81198778e+11, 1.96031299e+11, 2.11437784e+11, 2.26629261e+11,
           2.42091718e+11, 2.57165185e+11, 2.69583739e+11, 2.83035248e+11,
           2.97175729e+11] solMass>